In [26]:
import pandas as pd
import numpy as np
import statsmodels as stm
import scipy.stats
File_List = ['UCI_ionosphere_Normalized','UCI_iris_Normalized','UCI_wine_Normalized','UCI_glass_Normalized','UCI_parkinsons_Normalized','UCI_sonar-all_Normalized','UCI_vehicle_Normalized']

In [49]:
for file in File_List:
    filename = file+'.csv'
    DataFrame = pd.read_csv(filename)
    
    class_num = DataFrame['class'].max() + 1 
    #print(class_num)
    D_List = []
    S_List = []
    for feature in DataFrame:
        if feature == 'class' : continue
        mean_List = []
        std_List = []
        for i in range(class_num):
            s_by_class = DataFrame[feature][DataFrame['class'] == i]
            mean_List.append(s_by_class.mean())  
            std_List.append(s_by_class.std())
        #print(mean_List)
    #count D value
        D_value = 0
        for i in range(len(mean_List)):
            for j in range(len(mean_List)):
                if i == j : continue
                D_value += abs(mean_List[i]-mean_List[j])
        D_List.append(D_value)
    #count S value :
        S_J = np.std(std_List)
        S_value = S_J/(sum(std_List)/class_num)
        S_List.append(S_value)
    #print(D_List)
    #print(np.isnan(S_List[1]))
    #count C1,C2 matrix
    feature_num = len(D_List)
    C1 = np.eye(feature_num,feature_num)
    C2 = np.eye(feature_num,feature_num)
    for i in range(feature_num-1):
        for j in range(i+1,feature_num):
            if i == j : continue
            if D_List[i] == 0 :
                C1[i][j] = 0
                C1[j][i] = 99999
            elif D_List[j] == 0:
                C1[i][j] = 99999
                C1[j][i] = 0
            else:
                C1[i][j] = D_List[i] / D_List[j]
                C1[j][i] = 1/ C1[i][j]
                
            if S_List[i] == 0 or np.isnan(S_List[i]) :
                C2[i][j] = 0
                C2[j][i] = 99999
            elif S_List[j] == 0 or np.isnan(S_List[j]):
                C2[i][j] = 0
                C2[j][i] = 99999
            else:
                C2[i][j] = S_List[i] / S_List[j]
                C2[j][i] = 1/ C2[i][j]
    #print(C1,C2)       
    #normalize C1 ,C1
    #print((C2[:,1]))
    for j in range(feature_num):
        column_sum_1 = sum(C1[:,j])
        column_sum_2 = sum(C2[:,j])
        #print(column_sum_1)
        for i in range(feature_num):
            C1[i,j] = C1[i,j] / column_sum_1
            C2[i,j] = C2[i,j] / column_sum_2
    #print(C1,C2)       
    
    CV1 = []
    CV2 = []
    for i in range(feature_num):
        CV1.append(np.mean(C1[i]))
        CV2.append(np.mean(C2[i]))
    #print('CV1 : \n',CV1)
    #print('CV2 : \n',CV2)
    FC1 = np.matrix([CV1,CV2]).T
    FC2 = np.matrix([CV2,CV1]).T
    #print('FC \n',FC)
    #criteria preference by GC
    Nc = 2
    St = [np.std(CV1),np.std(CV2)]
    alpha = 0.01
    p_max = 1
    #print(FC)
    if feature_num >4 :
        try:
            res1 = stm.tsa.stattools.grangercausalitytests(FC1,p_max,verbose = False)
            res2 = stm.tsa.stattools.grangercausalitytests(FC2,p_max,verbose = False)
            F = [res1[1][0]['ssr_ftest'][0],res2[1][0]['ssr_ftest'][0]]
            cv = [scipy.stats.f.ppf(q=1-alpha, dfn=res1[1][0]['ssr_ftest'][3], dfd=res1[1][0]['ssr_ftest'][2]),scipy.stats.f.ppf(q=1-alpha, dfn=res2[1][0]['ssr_ftest'][3], dfd=res2[1][0]['ssr_ftest'][2])]
        except:
            F = [0,0]
            cv = [scipy.stats.f.ppf(q=1-alpha, dfn=1, dfd=0),scipy.stats.f.ppf(q=1-alpha, dfn=1, dfd=0)]
    
    else:
        F = [0,0]
        cv = [scipy.stats.f.ppf(q=1-alpha, dfn=1, dfd=0),scipy.stats.f.ppf(q=1-alpha, dfn=1, dfd=0)]
    
    #print(F,cv)
    P = np.eye(Nc,Nc)
    for i in range(0,Nc-1):
        Wp = 0
        for j in range(i+1,Nc):
            if St[i] > St[j]: Wp = St[i] / St[j]
            else: Wp = St[j] / St[i]
            #print(Wp)
            if F[i] > F[j] and F[i] > cv[i] :
                P[i,j] = Wp
                P[j,i] = 1/Wp
            else:
                P[j,i] = Wp
                P[i,j] = 1/Wp
    #normalized             
    for j in range(Nc):
        column_sum = sum(P[:,j])
        #print(column_sum)
        for i in range(Nc):
            P[i,j] = P[i,j] / column_sum
    #print(P)
    PV = [] 
    for i in range(Nc):
        PV.append(np.mean(P[i]))
    #print(PV)
    PV = np.matrix(PV)
    W = FC1 * PV.T
    #print('W',file,W.T,end = '\n\n')
    #print((W[0,0]))
    #print(W[2,0])
    
    Result_DataFrame = pd.DataFrame()
    for i,feature in enumerate(DataFrame):
        if feature == 'class':
            Result_DataFrame[feature] = DataFrame[feature]
        else:
            Result_DataFrame[feature] = DataFrame[feature].apply(lambda x : x*W[i,0])
    #print('DataFrame \n',DataFrame.head(3))
    #print('Result \n',Result_DataFrame.head(3))
    print(file)
    
    outfile_name = 'AHP_'+file.replace('_Normalized','')+'.csv'
    print(outfile_name)
    Result_DataFrame.to_csv(outfile_name,index = False)
    #break

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


UCI_ionosphere_Normalized
AHP_UCI_ionosphere.csv
UCI_iris_Normalized
AHP_UCI_iris.csv
UCI_wine_Normalized
AHP_UCI_wine.csv
UCI_glass_Normalized
AHP_UCI_glass.csv
UCI_parkinsons_Normalized
AHP_UCI_parkinsons.csv
UCI_sonar-all_Normalized
AHP_UCI_sonar-all.csv
UCI_vehicle_Normalized
AHP_UCI_vehicle.csv
